In [1]:
library(dplyr)
library(readxl)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [129]:
path <- "./data/results.xlsx"
df <- read_excel(path)

# Preprocessing

Remove the first row as it contained invalid response 🧌

In [130]:
df <- df[df$ID != 1,]

Regenerate row index 

In [131]:
df$ID <- 1:dim(df)[1]

Let's look at the columns

In [132]:
for (n in colnames(df)) {
    print(n)
}

[1] "ID"
[1] "Start time"
[1] "Completion time"
[1] "Email"
[1] "Name"
[1] "Wiek (w latach)"
[1] "Płeć"
[1] "Miejsce zamieszkania"
[1] "Wykształcenie"
[1] "Ile godzin dziennie spędzasz na Facebooku? (z wyłączeniem Messengera)"
[1] "Ile godzin dziennie spędzasz na Instagramie?"
[1] "Ile godzin dziennie spędzasz na TikToku?"
[1] "Czy w ostatnim miesiącu opublikował*ś post na którymś z powyższych social mediów?"
[1] "Uważam, że jestem osobą wartościową przynajmniej w takim samym stopniu, co inni."
[1] "Uważam, że posiadam wiele pozytywnych cech."
[1] "Ogólnie biorąc jestem skłonn* sądzić, że nie wiedzie mi się."
[1] "Potrafię robić różne rzeczy tak dobrze, jak większość innych ludzi."
[1] "Uważam, że nie mam wielu powodów, aby być z siebie dumn*."
[1] "Lubię siebie."
[1] "Ogólnie rzecz biorąc, jestem z siebie zadowolon*."
[1] "Chciał*bym mieć więcej szacunku dla samego siebie."
[1] "Czasami czuję się bezużyteczn*."
[1] "Niekiedy uważam, że jestem do niczego."


Rename the columns to something similer

In [133]:
df <- df %>%
  rename(
    age = "Wiek (w latach)",
    sex = "Płeć",
    education = "Wykształcenie",
    city = "Miejsce zamieszkania",
    facebookTime = "Ile godzin dziennie spędzasz na Facebooku? (z wyłączeniem Messengera)",
    instagramTime = "Ile godzin dziennie spędzasz na Instagramie?",
    tiktokTime = "Ile godzin dziennie spędzasz na TikToku?",
    publisher = "Czy w ostatnim miesiącu opublikował*ś post na którymś z powyższych social mediów?",
    ses1 = "Uważam, że jestem osobą wartościową przynajmniej w takim samym stopniu, co inni.",
    ses2 = "Uważam, że posiadam wiele pozytywnych cech.",
    ses3 = "Ogólnie biorąc jestem skłonn* sądzić, że nie wiedzie mi się.",
    ses4 = "Potrafię robić różne rzeczy tak dobrze, jak większość innych ludzi.",
    ses5 = "Uważam, że nie mam wielu powodów, aby być z siebie dumn*.",
    ses6 = "Lubię siebie.",
    ses7 = "Ogólnie rzecz biorąc, jestem z siebie zadowolon*.",
    ses8 = "Chciał*bym mieć więcej szacunku dla samego siebie.",
    ses9 = "Czasami czuję się bezużyteczn*.",
    ses10 = "Niekiedy uważam, że jestem do niczego."
  ) %>% 
  select(
    -c("Start time",
       "Completion time",
       "Name", 
       "Email"))

## Calculate SES score

In [134]:
toScore <- function(text, reverse = FALSE) {
    rawScore <- case_when(
        text == "zdecydowanie nie zgadzam się" ~ 1,
        text == "nie zgadzam się" ~ 2,
        text == "zgadzam się" ~ 3,
        text == "zdecydowanie zgadzam się" ~ 4,
        TRUE ~ NA
    )

    if (reverse) 5 - rawScore
    else rawScore
}

In [135]:
df <- df %>% 
 mutate(
   ses = toScore(ses1)
     + toScore(ses2)
     + toScore(ses3, TRUE)
     + toScore(ses4)
     + toScore(ses5, TRUE)
     + toScore(ses6)
     + toScore(ses7)
     + toScore(ses8, TRUE)
     + toScore(ses9, TRUE)
     + toScore(ses10, TRUE)
 ) 

We don't need the raw data anymore

In [136]:
df <- df %>% select(-ses1, -ses2, -ses3, -ses4, -ses5, -ses6, -ses7, -ses8, -ses9, -ses10)

## Convert the SM time to minutes

In [137]:
df <- df %>%  
  mutate(
      facebookTime = floor(as.numeric(facebookTime) * 60),
      instagramTime = floor(as.numeric(instagramTime) * 60),
      tiktokTime = floor(as.numeric(tiktokTime) * 60),
      socialMediaTime = facebookTime + instagramTime + tiktokTime
  ) %>%
  select(ID, age, sex, facebookTime, instagramTime, tiktokTime, socialMediaTime, publisher, ses)

## Divide participants in groups by SM percentiles

First, we'll divide participants into 3 groups based on their total time spent on social media:
1. Participants below the 33 percentile - low SM usage
2. Participants between 34and 66 percentile - moderate SM usage
3. Participants over 67 per percentile - high SM usage

In [138]:
quantile(df$socialMediaTime, c(0.33, 0.66))

33% 66% 
120 180

In [149]:
twoGroups <- function (quantiles) {
    function (time) {
        if (time <= quantiles[1]) "low"
        else "high"
    }
}

threeGroups <- function (quantiles) {
    function (time) {
        if (time <= quantiles[1]) "low"
        else if (time <= quantiles[2]) "moderate"
        else "high"
    }
}

In [150]:
q = c(0.33, 0.66)

df <- df %>% 
   mutate(
       facebookUsage = as.factor(sapply(facebookTime, threeGroups(quantile(df$facebookTime, q)))), 
       instagramUsage = as.factor(sapply(instagramTime, threeGroups(quantile(df$instagramTime, q)))), 
       tiktokUsage = as.factor(sapply(tiktokTime, threeGroups(quantile(df$tiktokTime, q)))),
       socialMediaUsage = as.factor(sapply(socialMediaTime, threeGroups(quantile(df$socialMediaTime, q)))),
       socialMediaUsageTwo = as.factor(sapply(socialMediaTime, twoGroups(quantile(df$socialMediaTime, 0.5)))),
   )

## Final DataFrame

In [151]:
glimpse(df)

Rows: 57
Columns: 14
$ ID                  <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,…
$ age                 <chr> "22", "21", "22", "21", "23", "22", "21", "19", "2…
$ sex                 <chr> "Mężczyzna", "Mężczyzna", "Kobieta", "Kobieta", "K…
$ facebookTime        <dbl> 0, 60, 60, 0, 0, 0, 180, 60, 0, 6, 60, 60, 60, 21,…
$ instagramTime       <dbl> 120, 120, 120, 60, 180, 60, 0, 60, 60, 60, 60, 120…
$ tiktokTime          <dbl> 0, 0, 0, 0, 0, 120, 0, 240, 0, 60, 0, 0, 0, 70, 60…
$ socialMediaTime     <dbl> 120, 180, 180, 60, 180, 180, 180, 360, 60, 126, 12…
$ publisher           <chr> "Nie", "Tak", "Nie", "Nie", "Nie", "Nie", "Nie", "…
$ ses                 <dbl> 31, 33, 33, 31, 24, 22, 29, 24, 18, 27, 20, 32, 29…
$ facebookUsage       <fct> low, moderate, moderate, low, low, low, high, mode…
$ instagramUsage      <fct> high, high, high, moderate, high, moderate, low, m…
$ tiktokUsage         <fct> low, low, low, low, low, high, low, high, low, hig…
$ socialMediaUsage 

### Liczności poszczególnych grup

In [156]:
table(df$socialMediaUsage)


    high      low moderate 
      16       30       11 

In [157]:
table(df[,c("socialMediaUsageTwo", "publisher")])

                   publisher
socialMediaUsageTwo Nie Tak
               high  15  12
               low   27   3

In [158]:
write.csv(df, "./output/data.csv", row.names = FALSE)

In [159]:
file.copy("./output/data.csv", "~/shared-vm/data.csv", overwrite = TRUE)

[1] TRUE